In [7]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse
import random

In [8]:
# Skip date
df1 = pd.read_csv('combined_data_1.txt', header = None, names = ['User_Id', 'Rating','Date'])
df2 = pd.read_csv('combined_data_2.txt', header = None, names = ['User_Id', 'Rating','Date'])
df3 = pd.read_csv('combined_data_3.txt', header = None, names = ['User_Id', 'Rating','Date'])
df4 = pd.read_csv('combined_data_4.txt', header = None, names = ['User_Id', 'Rating','Date'])
df = df1
df = df1.append(df2)
df = df.append(df3)
df = df.append(df4)

df['Rating'] = df['Rating'].astype(float)
df = df.drop('Date', 1)

print('Dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::5000000, :])#START:STOP:STEP (5000000)


Dataset shape: (100498277, 2)
-Dataset examples-
          User_Id  Rating
0              1:     NaN
5000000   2560324     4.0
10000000  2271935     2.0
15000000  1921803     2.0
20000000  1933327     3.0
941737    1465002     3.0
5941737    961023     4.0
10941737  1372532     5.0
15941737   854274     5.0
20941737   116334     3.0
25941737   768483     3.0
3959435   1331144     5.0
8959435   1609324     2.0
13959435  1699240     3.0
18959435  1776418     4.0
1353649   1643826     5.0
6353649    932047     4.0
11353649  2292868     4.0
16353649   932191     4.0
21353649  1815101     3.0
26353649   872339     4.0


In [9]:
df.index = np.arange(0,len(df))
# get movie count (where rating ==NaN)
movie_count = df.isnull().sum()[1] #df.isnull() searches for NaN values across all columns, we get in [1] column-rating

# get customer count (amount of unique user_id-movie_id)
user_count = df['User_Id'].nunique() - movie_count

# get rating count (for ratings not only unique, all values except movies ids)
rating_count = df['User_Id'].count() - movie_count

movie_count, user_count , rating_count

(17770, 480189, 100480507)

In [10]:
#Movie ID is really a mess import! 
#Looping through dataframe to add Movie ID column WILL make the Kernel run out of memory as it is too inefficient. 
#I achieve my task by first creating a numpy array with correct length 
#then add the whole array as column into the main dataframe!

#WORKING APPROXIMATELY 5 MIN

df_nan = pd.DataFrame(pd.isnull(df.Rating)) #new df with indexes standard and one column "Rating" with True/False values
#df1.Rating gets indexes+"Rating" column values
#pd.isnull(df1.Rating) gets indexes+"Rating" mask column values (True=NaN, False-otherwise)

df_nan = df_nan[df_nan['Rating'] == True]#new df with old indexes and one column "Rating" with only True values
df_nan = df_nan.reset_index() # adds new indexes, old-moves to new column "index"


 # zip(df_nan['index'][1:],df_nan['index'][:-1]) returns iterator of tuples

movie_np = []
movie_id = 1 #we can start with 1 and sum it in loop,because all movie_id sorted in dataset from 1 to last (increase order)

# i-current movie_id j-previous movie_id , its needed for counting the difference because
#we need to know amount of rows=amount of ratings 
# so (i-j-1)=amount of not null ratings of whole users dataset for one movie (which has movie_id)
for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), movie_id) # 1st arg-shape, second-value add row , because movie_np is array 
    temp
    movie_np = np.append(movie_np, temp) # adds element to the end of array
    movie_id += 1 #we can sum it in loop дшлу +1 ,because all movie_id sorted in dataset from 1 to last (increase order)
    
# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)
#finally after loop we get array movie_np size=amount of non null ratings, at each index it has corresponding 
#movie_id value for tis rating +can be concatenated with cleaned df
print('Length: {}'.format(len(movie_np)))

Length: 100480507


In [11]:
# remove those Movie ID rows from initial df (rows where rating=NaN)
df = df[pd.notnull(df['Rating'])]

df['Movie_Id'] = movie_np.astype(int) #Add movie_id column in df as movie_np received on previous step
df['User_Id'] = df['User_Id'].astype(int)

print('-Dataset examples-')
print(df.iloc[::5000000, :])



-Dataset examples-
           User_Id  Rating  Movie_Id
1          1488844     3.0         1
5000996     501954     2.0       996
10001962    404654     5.0      1962
15002876    886608     2.0      2876
20003825   1193835     2.0      3825
25004661   1899206     3.0      4661
30005496    154804     4.0      5496
35006274   2078749     5.0      6274
40007057    450763     5.0      7057
45007991    102092     3.0      7991
50009023    220298     5.0      9023
55010042    550530     5.0     10042
60011038    222570     3.0     11038
65011875   1273080     5.0     11875
70012676   2026970     5.0     12676
75013582    506044     4.0     13582
80014453    353605     2.0     14453
85015116    664606     3.0     15116
90016008   2213715     3.0     16008
95016879   1589401     5.0     16879
100017627  2314006     4.0     17627


In [14]:
df.to_csv('NetflixProcessedDataFile.csv')